<a href="https://colab.research.google.com/github/Tiabet/Project_Market/blob/master/AdditionalTraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [2]:
!pip install transformers
!pip install sentencepiece
!pip install keras_preprocessing
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.4 MB/s eta 0:00:00
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-y1txlna_/kobert-tokenizer_41534f4cbaba4b8fa2a0a4acb36e39a1
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-y1txlna_/kobert-tokenizer_41534f4cbaba4b8fa2a0a4acb36e39a1
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl

In [11]:
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

from kobert_tokenizer import KoBERTTokenizer

import csv
import torch
import pandas as pd
import random
import numpy as np

In [4]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [5]:
model_path = '/gdrive/MyDrive/Coda_Result/bert_model.pth'

device = "cuda" if torch.cuda.is_available() else "cpu"

model = BertForSequenceClassification.from_pretrained('skt/kobert-base-v1', num_labels=2).to(device)

model.load_state_dict(torch.load(model_path, map_location=device),strict=False)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at skt/kobert-base-v1 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


_IncompatibleKeys(missing_keys=[], unexpected_keys=['bert.embeddings.position_ids'])

In [15]:
add_train = ['만족합니다', '배송이 빠르고 좋아요', '좋아요', '매우 만족합니다', '꽤 만족합니다', '아주 좋습니다', '아주 좋아요', '좋습니다', '배송이 빨라요', '배송이 빠릅니다']
train_labels = [1] * len(add_train)

In [16]:
MAX_LEN = 128
BATCH_SIZE = 4

sentences = add_train
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
labels = [1]*len(add_train)

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

attention_masks = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

train_inputs = torch.tensor(input_ids)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(attention_masks)

train_data = TensorDataset(train_inputs, train_masks,train_labels)
train_sampler = SequentialSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

In [18]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 10

# 총 훈련 스텝
total_steps = len(train_dataloader) * epochs

# lr 조금씩 감소시키는 스케줄러
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [19]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)

        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행
        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask,
                        labels=b_labels)

        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))

print("")
print("Training complete!")


======== Epoch 1 / 10 ========
Training...

  Average training loss: 0.01

======== Epoch 2 / 10 ========
Training...

  Average training loss: 0.01

======== Epoch 3 / 10 ========
Training...

  Average training loss: 0.00

======== Epoch 4 / 10 ========
Training...

  Average training loss: 0.00

======== Epoch 5 / 10 ========
Training...

  Average training loss: 0.00

======== Epoch 6 / 10 ========
Training...

  Average training loss: 0.00

======== Epoch 7 / 10 ========
Training...

  Average training loss: 0.00

======== Epoch 8 / 10 ========
Training...

  Average training loss: 0.00

======== Epoch 9 / 10 ========
Training...

  Average training loss: 0.00

======== Epoch 10 / 10 ========
Training...

  Average training loss: 0.00

Training complete!


In [21]:
# Replace 'data.tsv' with the path to your TSV file
tsv_file = '/gdrive/MyDrive/Coda_Result/2199261531.tsv'

# Initialize an empty list to store the data from the TSV file
data = []

# Read the TSV file and store its content in the 'data' list
with open(tsv_file, 'r', newline='', encoding='utf-8') as file:
    reader = csv.reader(file, delimiter='\t')
    for row in reader:
        data.append(row)


In [22]:
df = pd.DataFrame(data[1:], columns=data[0])

In [23]:
reviews = df['review']

In [24]:
MAX_LEN = 128
BATCH_SIZE = 8

sentences = df['review']
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

attention_masks = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

test_inputs = torch.tensor(input_ids)
test_masks = torch.tensor(attention_masks)

test_data = TensorDataset(test_inputs, test_masks)
test_sampler = SequentialSampler(test_data)
real_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=BATCH_SIZE)

In [25]:
# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

pred_result = []
label = []

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(real_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(real_dataloader)))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)

    # 배치에서 데이터 추출
    b_input_ids, b_input_mask = batch

    # 그래디언트 계산 안함
    with torch.no_grad():
        # Forward 수행
        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask)

        # 로스 구함
    logits = outputs.logits

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    # Get predictions
    predictions = np.argmax(logits, axis=1)

    # Collect predictions and labels
    pred_result.extend(predictions.tolist())



In [26]:
df['pred_result'] = pred_result
df

,user_id,score,date,review,is_month,is_repurch,pred_result
0,sati*****,5,23.08.01.,진짜 귀얍게 잘 갖고 놀아요,True,False,1
1,kb****,5,23.07.03.,강아지 최애 공입니다. 노랑색을 유난히 좋아하네요,True,False,1
2,kare*****,5,23.05.24.,노견 푸들이 공놀이를 너무 좋아하는데 이빨이 약해서 탄탄한 공은 잘 못 물더라구요....,False,False,1
3,cm****,5,23.05.18.,저희 강아지가 진짜 좋아하는 공이라 여러개 샀는데 좋아요,True,False,1
4,meji****,5,22.11.10.,온것중에 빨강이 골라서 줬는데 환장하구 놀아욤 !!\n막 뜯어버리는 애도 아니고 짱...,False,False,1
...,...,...,...,...,...,...,...
63,haha****,5,17.10.28.,잘삿어요 이건 뜯지도 못하고 애들이 잘가지고 노네요 ㅎㅎ\n잘산거같아요ㅎㅎ 견당 하...,False,False,1
64,haha****,5,17.10.28.,만족합니다,False,False,1
65,hyun*****,5,17.10.12.,너무너무 좋아하네요 5분만에 삑삑이 고장내고도 계속 가지고 놀아요 ^^,False,False,1
66,ooy_***,5,17.10.20.,배송도 빠르고 강아지가 좋아해요^^,False,False,1


In [33]:
df['score'] = df['score'].replace({1: 0, 2: 0, '4': 1, '5': 1})

In [34]:
df[df['score']!=df['pred_result']]

,user_id,score,date,review,is_month,is_repurch,pred_result
0,sati*****,0,23.08.01.,진짜 귀얍게 잘 갖고 놀아요,True,False,1
1,kb****,0,23.07.03.,강아지 최애 공입니다. 노랑색을 유난히 좋아하네요,True,False,1
2,kare*****,0,23.05.24.,노견 푸들이 공놀이를 너무 좋아하는데 이빨이 약해서 탄탄한 공은 잘 못 물더라구요....,False,False,1
3,cm****,0,23.05.18.,저희 강아지가 진짜 좋아하는 공이라 여러개 샀는데 좋아요,True,False,1
4,meji****,0,22.11.10.,온것중에 빨강이 골라서 줬는데 환장하구 놀아욤 !!\n막 뜯어버리는 애도 아니고 짱...,False,False,1
...,...,...,...,...,...,...,...
63,haha****,0,17.10.28.,잘삿어요 이건 뜯지도 못하고 애들이 잘가지고 노네요 ㅎㅎ\n잘산거같아요ㅎㅎ 견당 하...,False,False,1
64,haha****,0,17.10.28.,만족합니다,False,False,1
65,hyun*****,0,17.10.12.,너무너무 좋아하네요 5분만에 삑삑이 고장내고도 계속 가지고 놀아요 ^^,False,False,1
66,ooy_***,0,17.10.20.,배송도 빠르고 강아지가 좋아해요^^,False,False,1


In [36]:
# Choose a folder path in your Google Drive to save the BERT model checkpoint
drive_folder_path = '/gdrive/MyDrive/Coda_Result/'

# Create the folder if it doesn't exist
import os
if not os.path.exists(drive_folder_path):
    os.makedirs(drive_folder_path)

# Save the model to the folder path in your Google Drive
model_path = drive_folder_path + 'bert_model_addition_trained.pth'
torch.save(model.state_dict(), model_path)
print("Save completed")

Save completed
